In [5]:
import json
import pandas as pd
from google.cloud import spanner
from typing import Any, Dict, List

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/adrian/PycharmProjects/NER-Pipeline/service-account-key.json'


PROJECT_ID   = "mptinc-playground"
INSTANCE_ID  = "tfgen-spanid-20250514170532172"
DATABASE_ID  = "vertex-pg-database"

spanner_client = spanner.Client(project=PROJECT_ID)
database       = spanner_client.instance(INSTANCE_ID).database(DATABASE_ID)

def spanner_to_df(sql: str,
                  params: dict | None = None,
                  param_types: dict | None = None) -> pd.DataFrame:
    with database.snapshot() as snap:
        it = snap.execute_sql(sql,
                              params=params or {},
                              param_types=param_types or {})
        rows = list(it)
        if not rows:
            return pd.DataFrame()
        cols = [f.name for f in it.metadata.row_type.fields]
        return pd.DataFrame(rows, columns=cols)


def _unwrap_to_dict(txt: str, max_depth: int = 3) -> Dict[str, Any] | None:
    cur: Any = txt
    for _ in range(max_depth):
        if isinstance(cur, str):
            try:            cur = json.loads(cur)
            except json.JSONDecodeError:  return None
        else: break
    return cur if isinstance(cur, dict) else None

def _explode_step_output(df_steps: pd.DataFrame) -> pd.DataFrame:
    rows: list[dict] = []

    for _, rec in df_steps.iterrows():
        obj = _unwrap_to_dict(rec["step_output"])
        if obj is None:
            continue

        common = {
            "verification_id":    rec["verification_id"],
            "dvs_correlation_id": rec["dvs_correlation_id"],
            "step_name":          rec.get("step_name"),
            "step_strategy":      rec.get("step_strategy"),
            "page":               rec.get("page"),
            "step_outcome":       rec.get("step_outcome"),
            "start_time":         rec.get("start_time"),
            "end_time":           rec.get("end_time"),
        }

        strategy = (rec.get("step_strategy") or rec.get("step_name") or "").upper()

        if strategy == "ENTITY_EXTRACTION":
            for field in obj.get("extractedFields", []):
                rows.append({**common, **field})

        elif strategy == "CLASSIFICATION":
            candidates = (
                    obj.get("classes")
                    or obj.get("predictions")
                    or [obj]
            )
            for cls in candidates:
                rows.append({**common, **cls})

        else:
            flat = pd.json_normalize(obj, sep=".").iloc[0].to_dict()
            rows.append({**common, **flat})

    return pd.DataFrame(rows)


def exploded_steps_for_correlation(corr_id: str) -> pd.DataFrame:
    sql_steps = """
                SELECT
                    s.*,
                    o.dvs_correlation_id
                FROM   `dvs_verification_steps`    AS s
                           JOIN   `dvs_verification_outcomes` AS o
                                  ON s.verification_id = o.verification_id
                WHERE  o.dvs_correlation_id = @corr_id \
                """
    params      = {"corr_id": corr_id}
    param_types = {"corr_id": spanner.param_types.STRING}

    steps = spanner_to_df(sql_steps, params, param_types)
    if steps.empty:
        return pd.DataFrame()

    return _explode_step_output(steps)

DVS_CORR_ID = "az-harness-122"
print("Connected DB:", database.name)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
exploded = exploded_steps_for_correlation(DVS_CORR_ID)
display(exploded.head(50))

Connected DB: projects/mptinc-playground/instances/tfgen-spanid-20250514170532172/databases/vertex-pg-database


E0000 00:00:1759321271.958228   47843 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
Created multiplexed session.
Created multiplexed session.
Created multiplexed session.
Created multiplexed session.
Created multiplexed session.


,verification_id,dvs_correlation_id,step_name,step_strategy,page,step_outcome,start_time,end_time,confidence,customerType,startTime,endTime,documentType,strategiesExecuted,classificationOutcome,value,label,entity,valid,riskLevel,gammaFlag,gammaScore,dpiFlag,dpiScore,dateFlag,dateScore,strategy,error,forgery,extractedFields,customerName,customerAddress,meterNumber
0,585b4d41-ef59-4d5a-a896-fbd3c57f38e1,az-harness-122,CLASSIFICATION,None,NaN,SUCCESS,2025-10-01 12:15:30.639945+00:00,2025-10-01 12:16:22.535056+00:00,0.95,PREPAID,"[2025, 10, 1, 14, 15, 30, 639945847]","[2025, 10, 1, 14, 16, 22, 535056275]",ELECTRICITY,[GEMINI_ONE_SHOT],CLASSIFIED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,585b4d41-ef59-4d5a-a896-fbd3c57f38e1,az-harness-122,ENTITY_EXTRACTION,None,0.0,SUCCESS,2025-10-01 12:15:24.171676+00:00,2025-10-01 12:15:30.618914+00:00,0.95,NaN,NaN,NaN,NaN,NaN,NaN,JOS ELECTRICITY DISTRIBUTION PLC,PROVIDER_ACRONYM,PROVIDER_ACRONYM,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,585b4d41-ef59-4d5a-a896-fbd3c57f38e1,az-harness-122,ENTITY_EXTRACTION,None,0.0,SUCCESS,2025-10-01 12:15:24.171676+00:00,2025-10-01 12:15:30.618914+00:00,0.99,NaN,NaN,NaN,NaN,NaN,NaN,"April, 2025",BILL_MONTH,BILL_MONTH,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,585b4d41-ef59-4d5a-a896-fbd3c57f38e1,az-harness-122,ENTITY_EXTRACTION,None,0.0,SUCCESS,2025-10-01 12:15:24.171676+00:00,2025-10-01 12:15:30.618914+00:00,0.99,NaN,NaN,NaN,NaN,NaN,NaN,632125213101,ACCOUNT_NUMBER,ACCOUNT_NUMBER,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,585b4d41-ef59-4d5a-a896-fbd3c57f38e1,az-harness-122,ENTITY_EXTRACTION,None,0.0,SUCCESS,2025-10-01 12:15:24.171676+00:00,2025-10-01 12:15:30.618914+00:00,0.99,NaN,NaN,NaN,NaN,NaN,NaN,MALL.KABIRU,CUSTOMER_NAME,CUSTOMER_NAME,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,585b4d41-ef59-4d5a-a896-fbd3c57f38e1,az-harness-122,ENTITY_EXTRACTION,None,0.0,SUCCESS,2025-10-01 12:15:24.171676+00:00,2025-10-01 12:15:30.618914+00:00,0.99,NaN,NaN,NaN,NaN,NaN,NaN,GOMBE ROAD BAUCHI,ADDRESS,ADDRESS,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,585b4d41-ef59-4d5a-a896-fbd3c57f38e1,az-harness-122,ENTITY_EXTRACTION,None,0.0,SUCCESS,2025-10-01 12:15:24.171676+00:00,2025-10-01 12:15:30.618914+00:00,0.99,NaN,NaN,NaN,NaN,NaN,NaN,0,PHONE_NUMBER,PHONE_NUMBER,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,585b4d41-ef59-4d5a-a896-fbd3c57f38e1,az-harness-122,ENTITY_EXTRACTION,None,0.0,SUCCESS,2025-10-01 12:15:24.171676+00:00,2025-10-01 12:15:30.618914+00:00,0.99,NaN,NaN,NaN,NaN,NaN,NaN,52.83,AMOUNT,AMOUNT,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,585b4d41-ef59-4d5a-a896-fbd3c57f38e1,az-harness-122,ENTITY_EXTRACTION,None,0.0,SUCCESS,2025-10-01 12:15:24.171676+00:00,2025-10-01 12:15:30.618914+00:00,0.99,NaN,NaN,NaN,NaN,NaN,NaN,"513,944.46",AMOUNT,AMOUNT,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,585b4d41-ef59-4d5a-a896-fbd3c57f38e1,az-harness-122,ENTITY_EXTRACTION,None,0.0,SUCCESS,2025-10-01 12:15:24.171676+00:00,2025-10-01 12:15:30.618914+00:00,0.99,NaN,NaN,NaN,NaN,NaN,NaN,"7,977.33",AMOUNT,AMOUNT,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
from google.cloud import spanner
import pandas as pd

import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/adrian/PycharmProjects/NER-Pipeline/service-account-key.json'
PROJECT_ID   = "mptinc-playground"
INSTANCE_ID  = "tfgen-spanid-20250514170532172"
DATABASE_ID  = "vertex-pg-database"

spanner_client = spanner.Client(project=PROJECT_ID)
database       = spanner_client.instance(INSTANCE_ID).database(DATABASE_ID)

def spanner_to_df(
        sql: str,
        params: dict | None = None,
        param_types: dict | None = None
) -> pd.DataFrame:
    """
    Run a SQL statement on Cloud Spanner and return the result as a DataFrame.
    """
    with database.snapshot() as snap:
        it = snap.execute_sql(
            sql,
            params=params or {},
            param_types=param_types or {},
        )
        rows = list(it)
        if not rows:
            return pd.DataFrame()

        cols = [f.name for f in it.metadata.row_type.fields]
        return pd.DataFrame(rows, columns=cols)

DVS_CORRELATION_ID = "az-harness-79"

sql_outcomes = """
               SELECT *
               FROM dvs_verification_outcomes
               WHERE dvs_correlation_id = @corr_id \
               """

params = {"corr_id": DVS_CORRELATION_ID}
param_types = {"corr_id": spanner.param_types.STRING}

dvs_verification_outcomes = spanner_to_df(sql_outcomes, params, param_types)

# (Optional) – fetch the matching steps as well
sql_steps = """
            SELECT *
            FROM dvs_verification_steps
            WHERE verification_id = "9ac27192-1651-4ffd-a2d2-19fe6d04dbf7" \
            """
dvs_verification_steps = spanner_to_df(sql_steps, params, param_types)

# Display
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

print("dvs_verification_outcomes")
display(dvs_verification_outcomes)

print("dvs_verification_steps")
display(dvs_verification_steps)

E0000 00:00:1759320746.052882   47843 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
Created multiplexed session.


dvs_verification_outcomes


,verification_id,dvs_correlation_id,document_reference,document_type,status,event_type,start_time,end_time,customer_id,customer_name,created_at,updated_at
0,db7e5619-8210-4369-b474-b0784891ece6,az-harness-79,2669443a-b94b-48ba-a889-48690b3dc891,ELECTRICITY,HITL Pending Review,VERIFICATION_FAILED_DVS,2025-09-04 12:45:57.122039+00:00,2025-09-04 12:47:20.247900+00:00,NGA-2025-07-26,TUKUR,2025-09-04 12:47:20.252874+00:00,2025-09-04 12:47:20.252873+00:00


dvs_verification_steps


,verification_id,step_id,step_name,start_time,end_time,step_outcome,step_output,created_at
0,9ac27192-1651-4ffd-a2d2-19fe6d04dbf7,42785719-8b41-4b92-8078-6f7deb6bd18d,EXTERNAL_VALIDATION,2025-09-29 13:08:06.245907+00:00,2025-09-29 13:08:06.246921+00:00,ERROR,"""{\""customerName\"":null,\""customerAddress\"":null,\""meterNumber\"":null,\""startTime\"":[2025,9,29,15,8,6,245907200],\""endTime\"":[2025,9,29,15,8,6,246921200],\""confidence\"":0.0,\""error\"":true,\""valid\"":false}""",2025-09-29 13:08:08.855904+00:00
1,9ac27192-1651-4ffd-a2d2-19fe6d04dbf7,435b8bfd-fb5c-42ed-9d14-3c91c8474936,CONTENT_VALIDATION,2025-09-29 13:08:06.261106+00:00,2025-09-29 13:08:06.278528+00:00,SUCCESS,"""{\""extractedFields\"":[{\""value\"":\""N36,000.00\"",\""confidence\"":0.98,\""label\"":\""AMOUNT\"",\""page\"":0,\""entity\"":\""AMOUNT\"",\""valid\"":true},{\""value\"":\""01/05/2025 - 01/06/2025\"",\""confidence\"":0.98,\""label\"":\""BILL_MONTH\"",\""page\"":0,\""entity\"":\""BILL_MONTH\"",\""valid\"":true},{\""value\"":\""19/05/2025\"",\""confidence\"":0.98,\""label\"":\""TRANSACTION_DATE\"",\""page\"":0,\""entity\"":\""TRANSACTION_DATE\"",\""valid\"":true},{\""value\"":\""AEDC\"",\""confidence\"":0.98,\""label\"":\""ELECTRICITY_COMPANY\"",\""page\"":0,\""entity\"":\""ELECTRICITY_COMPANY\"",\""valid\"":true},{\""value\"":\""Individual\"",\""confidence\"":0.98,\""label\"":\""CUSTOMER_TYPE\"",\""page\"":0,\""entity\"":\""CUSTOMER_TYPE\"",\""valid\"":true},{\""value\"":\""01/05/2025 - 01/06/2025\"",\""confidence\"":0.98,\""label\"":\""PERIOD\"",\""page\"":0,\""entity\"":\""PERIOD\"",\""valid\"":true},{\""value\"":\""SARAH RD\"",\""confidence\"":0.98,\""label\"":\""ADDRESS\"",\""page\"":0,\""entity\"":\""ADDRESS\"",\""valid\"":true},{\""value\"":\""SARAH RD\"",\""confidence\"":0.98,\""label\"":\""SERVICE_ADDRESS\"",\""page\"":0,\""entity\"":\""SERVICE_ADDRESS\"",\""valid\"":true},{\""value\"":\""709292004\"",\""confidence\"":0.98,\""label\"":\""ACCOUNT_NUMBER\"",\""page\"":0,\""entity\"":\""ACCOUNT_NUMBER\"",\""valid\"":true},{\""value\"":\""THOMAS DANSEL\"",\""confidence\"":0.98,\""label\"":\""CUSTOMER_NAME\"",\""page\"":0,\""entity\"":\""CUSTOMER_NAME\"",\""valid\"":true},{\""value\"":\""N35,412.65\"",\""confidence\"":0.98,\""label\"":\""AMOUNT\"",\""page\"":0,\""entity\"":\""AMOUNT\"",\""valid\"":true},{\""value\"":\""N32,942.00\"",\""confidence\"":0.98,\""label\"":\""AMOUNT\"",\""page\"":0,\""entity\"":\""AMOUNT\"",\""valid\"":true},{\""value\"":\""N2,470.65\"",\""confidence\"":0.98,\""label\"":\""AMOUNT\"",\""page\"":0,\""entity\"":\""AMOUNT\"",\""valid\"":true},{\""value\"":\""N595,671.91\"",\""confidence\"":0.98,\""label\"":\""AMOUNT\"",\""page\"":0,\""entity\"":\""AMOUNT\"",\""valid\"":true},{\""value\"":\""N36,000.00\"",\""confidence\"":0.98,\""label\"":\""AMOUNT\"",\""page\"":0,\""entity\"":\""AMOUNT\"",\""valid\"":true},{\""value\"":\""N559,671.91\"",\""confidence\"":0.98,\""label\"":\""AMOUNT\"",\""page\"":0,\""entity\"":\""AMOUNT\"",\""valid\"":true},{\""value\"":\""N595,084.56\"",\""confidence\"":0.98,\""label\"":\""AMOUNT\"",\""page\"":0,\""entity\"":\""AMOUNT\"",\""valid\"":true},{\""value\"":\""N36,000.00\"",\""confidence\"":0.98,\""label\"":\""AMOUNT\"",\""page\"":0,\""entity\"":\""AMOUNT\"",\""valid\"":true},{\""value\"":\""AEDC\"",\""confidence\"":0.98,\""label\"":\""PROVIDER_ACRONYM\"",\""page\"":0,\""entity\"":\""PROVIDER_ACRONYM\"",\""valid\"":true},{\""value\"":\""08039070070\"",\""confidence\"":0.98,\""label\"":\""PHONE_NUMBER\"",\""page\"":0,\""entity\"":\""PHONE_NUMBER\"",\""valid\"":true},{\""value\"":\""08152141414\"",\""confidence\"":0.98,\""label\"":\""PHONE_NUMBER\"",\""page\"":0,\""entity\"":\""PHONE_NUMBER\"",\""valid\"":true},{\""value\"":\""08152151515\"",\""confidence\"":0.98,\""label\"":\""PHONE_NUMBER\"",\""page\"":0,\""entity\"":\""PHONE_NUMBER\"",\""valid\"":true},{\""value\"":\""08152151515\"",\""confidence\"":0.98,\""label\"":\""PHONE_NUMBER\"",\""page\"":0,\""entity\"":\""PHONE_NUMBER\"",\""valid\"":true}]

In [11]:
import pandas as pd
import json

# Load the Excel file
df = pd.read_excel('DVS results.xlsx', sheet_name='Sheet1')

# Group by verification_id (column 3)
groups = df.groupby(df.columns[3])

# Function to normalize strings for comparison
def normalize(s):
    return str(s).strip().upper() if pd.notna(s) else ''

for ver_id, group in groups:
    print(f"\n### Verification ID: {ver_id}")

    # Customer info from first row (assuming consistent per group)
    first_row = group.iloc[0]
    account_num = normalize(first_row.iloc[12])
    name = ' '.join([normalize(first_row.iloc[13]), normalize(first_row.iloc[15]), normalize(first_row.iloc[14])]).strip()
    address = ' '.join([normalize(first_row.iloc[19]), normalize(first_row.iloc[20]), normalize(first_row.iloc[21]), normalize(first_row.iloc[22])]).strip()
    mobile = normalize(first_row.iloc[24])

    # Extracted fields
    extracted = {}
    for _, row in group.iterrows():
        step_name = row.iloc[5]
        if step_name in ['ENTITY_EXTRACTION', 'CONTENT_VALIDATION']:
            output_str = row.iloc[9]
            try:
                # Clean escaped quotes
                output_str = output_str.replace('\\"', '"')
                # Strip outer quotes if present
                if output_str.startswith('"') and output_str.endswith('"'):
                    output_str = output_str[1:-1]
                output = json.loads(output_str)
                fields = output.get('extractedFields', [])
                for field in fields:
                    label = field.get('label')
                    if label:
                        extracted[label] = {
                            'value': field.get('value'),
                            'confidence': field.get('confidence'),
                            'step': step_name
                        }
            except json.JSONDecodeError as e:
                print(f"Error parsing JSON for step {step_name} in {ver_id}: {str(e)}")

    account_extracted = extracted.get('ACCOUNT_NUMBER', {}).get('value')
    name_extracted = extracted.get('CUSTOMER_NAME', {}).get('value')
    address_extracted = extracted.get('ADDRESS', {}).get('value')
    phone_extracted = extracted.get('PHONE_NUMBER', {}).get('value')  # If available

    comp_data = {
        'Field': ['Account Number', 'Name', 'Address', 'Mobile Number'],
        'Customer Value': [account_num, name, address, mobile],
        'Extracted Value': [account_extracted, name_extracted, address_extracted, phone_extracted],
    }
    pd.set_option("display.max_colwidth", None)
    comp_df = pd.DataFrame(comp_data)
    display(comp_df)


### Verification ID: 00058e80-b548-4046-b896-a1845390a042


,Field,Customer Value,Extracted Value
0,Account Number,18465603,30630167499
1,Name,UMMA MOHAMMED,IDI ALI
2,Address,"30 ROAD A MALALI NEW EXTENSION , KADUNA NORTH, KADUNA ROAD A MALALI NEW EXTENSION KADUNA NORTH KADUNA",NO 30 ROAD A LAWL AMINU\nSTREET KADUNA
3,Mobile Number,8051111720,None



### Verification ID: 001df456-bb54-44f2-8844-b8cb7a848b57


,Field,Customer Value,Extracted Value
0,Account Number,4872448,0000951281
1,Name,AFEEZ ODEWUMI,Segun Aremu
2,Address,"12 OVIA SOUTH, OKOKPON, EDO, NIGERIA OVIA SOUTH EDO",45 Ukonkpon TOWN Ukonkpon TOWN
3,Mobile Number,2349165704156,08000000000



### Verification ID: 0025095b-3fbc-4a60-801d-411cc1d9db32


...

,Field,Customer Value,Extracted Value
0,Account Number,7305143,260237830751
1,Name,SALIHU HAMISU,SANI LAWALI
2,Address,"SALIHI HAMISU, UNIT 34, NTA ROAD, ESAN WEST, EKPOMA, 310103, EDO, NIGERIA NTA ROAD ESAN WEST EDO",DOGON KADE VILLAGE AREA
3,Mobile Number,2347039989029,None



### Verification ID: 05c31d1f-bbda-4e70-a080-e962dd2d9022


,Field,Customer Value,Extracted Value
0,Account Number,27579887,28/39/10/0726-01
1,Name,EMMANUEL AGEVA,ADEWALE WASIU
2,Address,"OSUNE, UNIT FOOTBALL , 08142184813 IKIRE.OSUNE STATE , IREWOLE, 221104, NIGERIA IKIRE.OSUNE STATE IREWOLE OSUN",SACBA'S COMP IK RE
3,Mobile Number,8142184813,09062924605



### Verification ID: 05c422d7-8641-40f6-9830-1144297f4ad6


,Field,Customer Value,Extracted Value
0,Account Number,1858163,0641-5341-3826-7683-9077
1,Name,TOBI BABALOLA,OMONIYI ADENLE YOMI
2,Address,"SANNI BALOGUN STREET, 71 , IFAKO/IJAYE, LAGOS, 101232, NIGERIA 71 IFAKO/IJAYE LAGOS","71 SANNI BALOGUN STREET\nFAGBA,LAGOS"
3,Mobile Number,2349035668104,None


In [2]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
